### advent callendar
##### Analysis

- KAM topicのクラスタリング
 テキストクラスタリング(e5)
 llm
- KAM topicの時系列分析
- KAM topicと業種
- 勘定科目からの予測
- スモールデータセットの提供(2021,2022,2023,2024の6月)業種を絞る リポジトリはベータとして提供
     - KAM項目の抽出
     - リスクテキスト項目の抽出
     - 会計数値の抽出

# TODO

jppfsのpreがない
bs_dfが0になる

# Get metadata

# setup

In [ ]:
from time import sleep
from tqdm import tqdm
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
DATA_PATH=Path("/Users/yusuke_noro/Documents/Projects/edinet_xbrl_prep/data")



In [2]:
from edinet_xbrl_prep.edinet_api import request_term, edinet_response_metadata, request_doc

# query edinet

In [3]:
res_results = request_term(api_key=your_api_key, start_date_str='2024-06-01', end_date_str='2024-06-05')

In [ ]:
edinet_response_metadata_obj=edinet_response_metadata()
edinet_response_metadata_obj.set_data(res_results)
filename = str(DATA_PATH/"data.jsonl")
edinet_response_metadata_obj.save(filename)

In [3]:

edinet_response_metadata_obj=edinet_response_metadata(
    filename = DATA_PATH/"data.jsonl",
    tse_sector_url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls",
    tmp_path = DATA_PATH
    )

yuho_df = edinet_response_metadata_obj.get_yuho_df()

In [4]:
#yuho_df.sector_label_33.value_counts()

In [5]:
yuho_df_filtered = yuho_df.query("sector_label_33 == '食料品'")

# Load

In [6]:
download_docid_list = yuho_df_filtered.docID.to_list()

In [6]:
res_results = []
(DATA_PATH / "xbrl_doc").mkdir(parents=True, exist_ok=True)
for docid in tqdm(download_docid_list):
    out_filename = DATA_PATH / "xbrl_doc" / (docid + ".zip")
    res_results.append(request_doc(api_key=your_api_key, docid=docid, out_filename_str=out_filename))
    sleep(0.5)
#out_filename = PROJPATH+"data/0_metadata/trial/res_results_doc_teisei_yuho_2407_v0830.pkl.cmp"
#joblib.dump(res_results_teisei_yuho, out_filename, compress=True) 

100%|██████████| 79/79 [03:34<00:00,  2.72s/it]


# Link

In [7]:
from edinet_xbrl_prep.link_base_file_analyzer import account_list_common, fs_tbl_loader

In [9]:

account_list_common_obj = account_list_common(
    data_path=DATA_PATH,
    account_list_year="2024"
)
(DATA_PATH / "xbrl_doc_ext").mkdir(parents=True, exist_ok=True)



label:  844
label:  5073


In [10]:
import sys
arelle_temp_dir = Path(DATA_PATH/"tmp")
arelle_dir = arelle_temp_dir.joinpath('arelle_tmp')
arelle_dir.mkdir(exist_ok=True)
cmd_arelle_dir = '--xdgConfigHome=' + str(arelle_dir)
sys.argv.append(cmd_arelle_dir)

In [11]:
#from edinet_xbrl_prep.xbrl_parser_rapper import xbrl_parser_rapper

#### 財務数値の取り出し
有価証券報告書XBRLから財務数値を取得します。

In [12]:

#docid="S100TLB3"
fs_tbl_loader_obj_list = []
for docid in tqdm(download_docid_list[1:3]):
    fs_tbl_loader_obj = fs_tbl_loader(
        account_list_common_obj=account_list_common_obj,
        docid=docid,
        zip_file_str=DATA_PATH / "raw/xbrl_doc" / (docid + ".zip"),
        temp_path_str=DATA_PATH / "raw/xbrl_doc_ext" / docid,
        role_label_list=['BS', 'PL', 'report'],
    )
    fs_tbl_loader_obj_list.append(fs_tbl_loader_obj)



  0%|          | 0/2 [00:00<?, ?it/s]

[read_linkbase_file] done in 1.91 s 
org 20
common: 2570
[make_account_tbl] done in 0.00 s 
[make_account_link_tracer] done in 0.00 s 
[make_account_tbl_role_dict] done in 8.76 s 


 50%|█████     | 1/2 [00:20<00:20, 20.39s/it]

[get_xbrl_rapper] done in 9.66 s 
[read_linkbase_file] done in 3.21 s 
org 37
common: 2570
[make_account_tbl] done in 0.00 s 
[make_account_link_tracer] done in 0.00 s 
[make_account_tbl_role_dict] done in 12.82 s 


100%|██████████| 2/2 [00:47<00:00, 23.51s/it]

[get_xbrl_rapper] done in 10.52 s 


In [13]:
import pandas as pd

In [14]:
bs_df_list = []
for itr_fs_tbl_loader in fs_tbl_loader_obj_list:
    bs_df = itr_fs_tbl_loader.get_data(
        doc_name='report',
        term='current',
        Consolidated=True
        )
    bs_df_list.append(bs_df)
bs_df = pd.concat(bs_df_list).reset_index(drop=True)

AssertionError: doc_name should be one of ['BS','PL','CF','SS','NOTES']

In [ ]:
def preproc(org_data):
    #org_data=org_data.copy()
    #org_data['data'] = org_data['data'].str.replace(',','')
    org_data['data'] = pd.to_numeric(org_data['data_str'], errors='coerce')
    org_data['data'] = org_data['data'].fillna(0)
    org_data['data'] = org_data.data.astype('int') # for case of data is string object (cannot read csv with dtype=int)
    org_data=org_data.assign(
        #key_tag=org_data.key_org.str.replace(':','_'),
        context_ref_len=(org_data.context_ref
                         .str.split('_',expand=True)
                         .notna().sum(axis=1) # count component separated by '_'
                         ),
        #data_credit=org_data.data
        )
    assert org_data.reset_index()[['key','context_ref']].duplicated().sum()==0
    #self.org_data=org_data
    return org_data

bs_df=preproc(bs_df)#.head(20)

In [19]:

bs_df2=bs_df.sort_values('context_ref_len',ascending=True).drop_duplicates(keep='first',subset='key')

#bs_df2=bs_df2.assign(data_credit=((bs_df2.balance=='credit')*2-1)*bs_df2.data)

#### テキストの取り出し

In [33]:
fs_tbl_loader_obj_list[1].linkbasefile_obj.parent_child_df.role.value_counts()
#.query("role.str.contains('rol_Audits')")
#get_presentation_account_list_obj.export_label_to_taxonomi_dict()

role
http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_CabinetOfficeOrdinanceOnDisclosureOfCorporateInformationEtcFormNo3AnnualSecuritiesReport    196
http://disclosure.edinet-fsa.go.jp/role/jppfs/rol_BalanceSheet                                                                                 82
http://disclosure.edinet-fsa.go.jp/role/jppfs/rol_ConsolidatedBalanceSheet                                                                     81
http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_Shareholdings-02                                                                             69
http://disclosure.edinet-fsa.go.jp/role/jppfs/rol_ConsolidatedStatementOfCashFlows-indirect                                                    50
http://disclosure.edinet-fsa.go.jp/role/jppfs/rol_ConsolidatedStatementOfIncome                                                                44
http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_InformationAboutEmployees-05                                         

In [16]:
pd.pivot_table(index='docID',columns='label',values='value',aggfunc='first')

TypeError: pivot_table() missing 1 required positional argument: 'data'

In [3]:
linkbasefile_obj=linkbasefile(
    zip_file_str=DATA_PATH / "xbrl_doc" / "S100TLB3.zip",
    temp_path_str=DATA_PATH / "xbrl_doc_ext"/"S100TLB3")
linkbasefile_obj.read_linkbase_file()
linkbasefile_obj.check()
linkbasefile_obj.make_account_label(account_list_common_obj)
#linkbasefile_obj.make_summary_tbl()
#linkbasefile_obj.make_account_label_common(account_list_common_obj) 


152
379
529


## analysis link

In [20]:
account_link_tracer_obj.search_keys(keyword_in_taxonomy='Sales')

['jpcrp_cor:NetSalesSummaryOfBusinessResults',
 'jppfs_cor:NetSalesAbstract',
 'jppfs_cor:NetSales',
 'jppfs_cor:CostOfSales',
 'jppfs_cor:CostOfSalesAbstract']

In [16]:
account_link_tracer_obj.get_child_keys_recursive(
    parent_key='jppfs_cor:NetSalesAbstract',
    role=role_text)

['jpcrp030000-asr_E03926-000:ArentIncome', 'jppfs_cor:NetSales']

# xbrl parse

In [46]:
docid="S100TLB3"
xbrl_data_df,log_dict=get_xbrl_rapper(
    docid=docid,
    zip_file=DATA_PATH / "xbrl_doc" / (docid + ".zip"),
    temp_dir=DATA_PATH / "tmp",
    out_path=DATA_PATH / "xbrl_doc_ext"/"S100TLB3",
    update_flg=False)


In [49]:
pl_key=linkbasefile_obj.account_tbl_role_dict['rol_BusinessResultsOfReportingCompany'].key

In [53]:
tmp=pd.merge(
    linkbasefile_obj.account_tbl_role_dict['rol_BusinessResultsOfReportingCompany'],
    xbrl_data_df.query("key in @pl_key"),
    on='key',
    how='outer')#.query("not key.str.contains('Abstract')")


In [ ]:
dict_add,log_dict=get_data(
    account_link_tracer_obj=account_link_tracer_obj,
    xbrl_obj=textblock_cur,
    include_tree_top_keyword=conf['include_tree_top_keyword'],
    role_text=role_text,
    include_keyword_list=conf['include_keyword_list'],
    exclude_keyword=conf['exclude_keyword'],
    keep_taxonomi_list=conf['keep_taxonomi_list'],
    docid=docid,
    edinet_code=edinet_code,
    period_end=period_end,
    period_start=period_start,
    tag=conf['tag'],
    log_dict=log_dict
    )